In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False 
cache_dir = "/data1/debajyoti/test/llms/models"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Telugu-LLM-Labs/Indic-gemma-7b-finetuned-sft-Navarasa-2.0",
    cache_dir = cache_dir,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map="auto"
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

input_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

input_text = input_prompt.format(
        """You are a humor recognition assistant judging if an 'Input' is humorous or not. Dont give output anything else.
    If the 'Input' is humorous, you need to output your final 'Output' as 'Humor'. 
    If the 'Input' is non-humorous, you need to output your final 'Output' as 'Non-humor'.""", # instruction
        "Chalo protocol pata chal gaya. Vice President agar Muslim na hote toh shayad kabhi pata hi nahi chaltapic.twitter.com/bOLAEk0g4E", # input
        "", # output - leave this blank for generation!
    )

inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
response = tokenizer.batch_decode(outputs)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/data1/debajyoti/test/llms/envllm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
response

["<bos>\n### Instruction:\nYou are a humor recognition assistant judging if an 'Input' is humorous or not. Dont give output anything else.\n    If the 'Input' is humorous, you need to output your final 'Output' as 'Humor'. \n    If the 'Input' is non-humorous, you need to output your final 'Output' as 'Non-humor'.\n\n### Input:\nChalo protocol pata chal gaya. Vice President agar Muslim na hote toh shayad kabhi pata hi nahi chaltapic.twitter.com/bOLAEk0g4E\n\n### Response:\nNon-humor<eos>"]

In [ ]:
# input_text = input_prompt.format(
#         """You are a humor recognition assistant judging if an 'Input' is humorous or not. Dont give output anything else.
#     If the 'Input' is humorous, you need to output your final 'Output' as 'Humor'. 
#     If the 'Input' is non-humorous, you need to output your final 'Output' as 'Non-humor'.""", # instruction
#         "What happens if Usain Bolt misses his bus? He waits for it at the next stop.", # input
#         "", # output - leave this blank for generation!
#     )

# inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
# response = tokenizer.batch_decode(outputs)
# response

In [ ]:
import pandas as pd
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# train_df_eng = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/1Humour_English(NEW).csv")
# train_df_eng = train_df_eng[:2000]
# train_df_hin = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/1Humour_English(NEW).csv")
# train_df_hin = train_df_hin[:2000]
# train_df = pd.concat([train_df_eng, train_df_hin], ignore_index=True)
# train_df = train_df_hin
train_df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/splits/humor/train.csv")
val_df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/splits/humor/val.csv")
test_df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/splits/humor/test.csv")

test_df

,Sentence,Tag
0,Is 2 takiye ki naukri main mera lakhon ka worl...,0
1,"""Naam : Viv Richards\nBaap ka naam : Master Di...",1
2,Sakshi Maharaj is BJP's Digvijaya Singh.,1
3,Aaj @British_Airways ki toh lag gayi bhai.\nHa...,1
4,Ghanta development hoga! Saare paise to electi...,1
...,...,...
291,"Hum to Aam Aadmi hain ji, woh Megalomaniac hai...",1
292,Neend nahi aati hai raaton mein?,0
293,Ghar mein Bipasha Basu ki photo rakhne se bhoo...,1
294,Aur dikhao aur dikhaopic.twitter.com/Ij4R6xNg9V,0


In [ ]:
df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/outputs/humor/few_shots/colbert_eng_llama.csv")
df

,Sentence,Tag
0,दिमाग पैराशूट की तरह हैं।वे खुले होने पर सबसे ...,Humor
1,"राजकुमारी, राजकुमारों के साथ पर्याप्त अनुभव था...",Non-humor
2,"यदि यह पर्यटक का मौसम है, तो मुझे लाइसेंस कहां...",Humor
3,"यदि कोई जॉगर ध्वनि की गति से चलता है, तो क्या ...",Non-humor
4,"जब आपका जीवन एक लाख टुकड़ों में बिखर जाता है, ...",Humor
5,"मेरी प्रेमिका ने सोचा कि यह सिर्फ एक मसौदा है,...",Non-humor
6,आपने मुझे किसी ऐसे व्यक्ति के लिए गलत किया है ...,Humor
7,"बस जब मैं सिरों को पूरा करता हूं, तो कोई छोर ल...",Non-humor


In [ ]:
# Convert dataframe to specified format
formatted_few_shot_examples = []
for _, row in df.iterrows():
    formatted_few_shot_examples.append({'role': 'user', 'content': row['Sentence']})
    formatted_few_shot_examples.append({'role': 'assistant', 'content': row['Tag']})
    
formatted_few_shot_examples

[{'role': 'user',
  'content': 'दिमाग पैराशूट की तरह हैं।वे खुले होने पर सबसे अच्छा काम करते हैं।बस सुनिश्चित करें कि तार अभी भी संलग्न हैं।'},
 {'role': 'assistant', 'content': 'Humor'},
 {'role': 'user',
  'content': 'राजकुमारी, राजकुमारों के साथ पर्याप्त अनुभव था, मेंढक की तलाश करता है।'},
 {'role': 'assistant', 'content': 'Non-humor'},
 {'role': 'user',
  'content': 'यदि यह पर्यटक का मौसम है, तो मुझे लाइसेंस कहां मिलेगा?'},
 {'role': 'assistant', 'content': 'Humor'},
 {'role': 'user',
  'content': 'यदि कोई जॉगर ध्वनि की गति से चलता है, तो क्या वह अभी भी अपने वॉकमैन को सुन सकता है?'},
 {'role': 'assistant', 'content': 'Non-humor'},
 {'role': 'user',
  'content': 'जब आपका जीवन एक लाख टुकड़ों में बिखर जाता है, तो टुकड़ों को उठाएं, कुछ गोंद को पकड़ें, और एक नया बनाएं।'},
 {'role': 'assistant', 'content': 'Humor'},
 {'role': 'user',
  'content': 'मेरी प्रेमिका ने सोचा कि यह सिर्फ एक मसौदा है, इसलिए उसने किनारों को स्कैलप किया और इसे कपकेक लाइनर के लिए इस्तेमाल किया।'},
 {'role': 'assist

In [ ]:
formatted_few_shot_examples = ""
for index, row in df.iterrows():
    if index == 0 or index == 4:
        formatted_few_shot_examples += f"""'Input': "{row['Sentence']}"\n'Response':"{row['Tag']}"\n\n"""
    # formatted_few_shot_examples += f"""'Input': "{row['Sentence']}"\n'Response':"{row['Tag']}"\n\n"""
    # if index == 1:
    #     break

# Display the result
print(formatted_few_shot_examples)

'Input': "दिमाग पैराशूट की तरह हैं।वे खुले होने पर सबसे अच्छा काम करते हैं।बस सुनिश्चित करें कि तार अभी भी संलग्न हैं।"
'Response':"Humor"

'Input': "जब आपका जीवन एक लाख टुकड़ों में बिखर जाता है, तो टुकड़ों को उठाएं, कुछ गोंद को पकड़ें, और एक नया बनाएं।"
'Response':"Humor"




### Create prompt, parse response and  generate response

In [ ]:
from transformers import pipeline
import re
import json

# Define the function to create the prompt
def create_prompt(input, num_examples):

    # System prompt 
    system_prompt = """You are a humor recognition assistant judging if an 'Input' is sarcastic or not. 
        If the 'Input' is sarcastic, you need to give your final 'Output' as "Humor". 
        If the 'Input' is non-sarcastic, you need to give your final 'Output' as "Non-humor". Dont output anything else. """

    messages = [system_prompt + formatted_few_shot_examples, input]
    # messages = [system_prompt, input]

    return messages

def parse_response(response):
    """
    Extracts the classification label and explanation from the model-generated response,
    starting from the line containing the final "Answer:".

    Args:
        response (str): The response generated by the model.

    Returns:
        tuple: A tuple containing the classification label ("Humor" or "Non-humor") 
               and the explanation text.
    """
    # print(response)
    if len(response) > 0:        
        if "Non-humor" in response:
            return "0"
        elif "Humor" in response:
            return "1"
        else:# If no label is found, return None
            return None

def generate_responses(prompt):
    
    input_prompt = """
    ### Instruction:
    {}

    ### Input:
    {}

    ### Response:
    {}"""

    input_text = input_prompt.format(
            f"""{prompt[0]}""", # instruction
            f"{prompt[1]}", # input
            "", # output - leave this blank for generation!
        )

    inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 400, use_cache = True)
    response = tokenizer.batch_decode(outputs)
    
    return response

In [ ]:
def process_data(output_filepath):
    # df = pd.read_csv(data_filepath)
    df = test_df
    # df = df[:5]
    
    
    results = []
    num_examples = 15 # number of shots in few-shot
    # Loop over the DataFrame and pass the text to generate response
    for index, row in tqdm(list(df.iterrows())):
        prompt = create_prompt(row['Sentence'], num_examples)
        model_response = generate_responses(prompt)
        # print(model_response)
        generated_label = parse_response(model_response)

        # veracity, explanation = parse_response(model_response)
        
        results.append({
            'Sentence': row['Sentence'],
            'Label': row['Tag'],
            'Response': model_response,
            'Gen_label': generated_label,
            # 'Explanation': explanation
        })
        # break
    
    results = pd.DataFrame(results)
    
    return results

# Example file paths, replace with your actual paths
output_filepath = '/data1/debajyoti/test/llms/data/humor'

# Run the data processing
result = process_data(output_filepath)

  0%|          | 0/296 [00:00<?, ?it/s]

100%|██████████| 296/296 [01:11<00:00,  4.14it/s]


In [ ]:
result

,Sentence,Label,Response,Gen_label
0,Is 2 takiye ki naukri main mera lakhon ka worl...,0,[<bos>\n ### Instruction:\n You are a hu...,None
1,"""Naam : Viv Richards\nBaap ka naam : Master Di...",1,[<bos>\n ### Instruction:\n You are a hu...,None
2,Sakshi Maharaj is BJP's Digvijaya Singh.,1,[<bos>\n ### Instruction:\n You are a hu...,None
3,Aaj @British_Airways ki toh lag gayi bhai.\nHa...,1,[<bos>\n ### Instruction:\n You are a hu...,None
4,Ghanta development hoga! Saare paise to electi...,1,[<bos>\n ### Instruction:\n You are a hu...,None
...,...,...,...,...
291,"Hum to Aam Aadmi hain ji, woh Megalomaniac hai...",1,[<bos>\n ### Instruction:\n You are a hu...,None
292,Neend nahi aati hai raaton mein?,0,[<bos>\n ### Instruction:\n You are a hu...,None
293,Ghar mein Bipasha Basu ki photo rakhne se bhoo...,1,[<bos>\n ### Instruction:\n You are a hu...,None
294,Aur dikhao aur dikhaopic.twitter.com/Ij4R6xNg9V,0,[<bos>\n ### Instruction:\n You are a hu...,None


In [ ]:
from pydantic import BaseModel, validator
import re

class ModelResponse(BaseModel):
    instruction: str
    input_text: str
    output_label: str

    # @validator('instruction', pre=True, always=True)
    # def extract_instruction(cls, value, values):
    #     # Extract the Instruction part
    #     match = re.search(r'### Instruction:\n(.+?)\n\n### Input:', value, re.DOTALL)
    #     return match.group(1).strip() if match else ""

    # @validator('input_text', pre=True, always=True)
    # def extract_input_text(cls, value, values):
    #     # Extract the Input part
    #     match = re.search(r'### Input:\n(.+?)\n\n### Response:', value, re.DOTALL)
    #     return match.group(1).strip() if match else ""

    @validator('output_label', pre=True, always=True)
    def extract_output_label(cls, value, values):
        # Extract the Response part
        match = re.search(r'### Response:\n(.+?)<eos>', value, re.DOTALL)
        return match.group(1).strip() if match else ""

# Example usage
response_text = (
    "<bos>\n### Instruction:\nYou are a humor recognition assistant judging if an 'Input' "
    "is humorous or not. Dont give output anything else.\n    If the 'Input' is humorous, "
    "you need to output your final 'Output' as 'Humor'. \n    If the 'Input' is non-humorous, "
    "you need to output your final 'Output' as 'Non-humor'.\n\n### Input:\nWhat happens if Usain Bolt "
    "misses his bus? He waits for it at the next stop.\n\n### Response:\nHumor<eos>"
)

# Create ModelResponse object
parsed_response = ModelResponse(instruction=response_text, input_text=response_text, output_label=response_text)

# Access parsed fields
# print("Instruction:", parsed_response.instruction)
# print("Input Text:", parsed_response.input_text)
print("Output Label:", parsed_response.output_label)

Output Label: Humor


/tmp/ipykernel_1120943/869760653.py:21: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('output_label', pre=True, always=True)


In [ ]:
for index, row in tqdm(list(result.iterrows())):
    parsed_response = ModelResponse(instruction=row['Response'][0], input_text=row['Response'][0], output_label=row['Response'][0])

    # Access parsed fields
    print("Output Label:", parsed_response.output_label)
    result['Gen_label'][index] = parsed_response.output_label
    print(result['Gen_label'][index])

In [ ]:
result['Gen_label'].value_counts()

In [ ]:
# Standardize the Gen_label column values
result['Gen_label'] = result['Gen_label'].str.replace(r'\W', '', regex=True).str.lower()

# Map the standardized labels to the final desired labels
result['Gen_label'] = result['Gen_label'].map({
    'nonhumor': 'Non-humor',
    'humor': 'Humor'
})

# Check the updated value counts
print(result['Gen_label'].value_counts())

In [ ]:
# Map string labels to integers
label_mapping = {'Non-humor': 0, 'Humor': 1}
new_result['Gen_label'] = new_result['Gen_label'].map(label_mapping)

# Check the result
print(new_result)

In [ ]:
y_true = list(map(int, new_result['Label']))
y_pred = list(map(int, new_result['Gen_label']))
print(classification_report(y_true, y_pred, digits=6))